In [1]:
# gpu using
import tensorflow as tf
from keras import backend as K
if len(K.tensorflow_backend._get_available_gpus()) > 0:
    config = tf.ConfigProto(device_count = {'GPU': 1})
    print('Using gpu')
else:
    config = tf.ConfigProto(device_count={"CPU": 32},
                            inter_op_parallelism_threads=32,
                            intra_op_parallelism_threads=32)
K.set_session(K.tf.Session(config=config))    

Using TensorFlow backend.


Using gpu


In [2]:
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense, Activation, Conv2D, MaxPool2D, Dropout, Flatten

# Loading Data with augmentation

In [79]:
datagen = ImageDataGenerator(rescale=1. / 255, rotation_range=20, 
                              width_shift_range=5.0,
                              height_shift_range=5.0,
                              shear_range=5.0,
                              featurewise_center=False,
                              zoom_range=[0.8, 1.2],
                              fill_mode='nearest',
                              horizontal_flip=True,
                              vertical_flip=False,
                              featurewise_std_normalization=False)

train_gen= datagen.flow_from_directory(
    'train',
    target_size=(150, 150),
    batch_size=64,
    class_mode='binary')
val_gen = datagen.flow_from_directory(
    'val',
    target_size=(150, 150),
    batch_size=64,
    class_mode='binary')

Found 24000 images belonging to 2 classes.
Found 1000 images belonging to 2 classes.


# CNN model

In [4]:
model = Sequential()
model.add(Conv2D(32, (3, 3), input_shape=(150, 150, 3)))
model.add(Activation('relu'))
model.add(MaxPool2D(pool_size=(2, 2)))

model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPool2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPool2D(pool_size=(2, 2)))

model.add(Flatten())
model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(1))
model.add(Activation('sigmoid'))

In [5]:
model.compile(optimizer='adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

In [6]:
batch_size = 64
# Количество изображений для обучения
nb_train_samples = 24000
# Количество изображений для проверки
nb_validation_samples = 1000

model.fit_generator(
    train_gen,
    steps_per_epoch=int(nb_train_samples // batch_size),
    epochs=30,
    validation_data=val_gen,
    validation_steps=int(nb_validation_samples // batch_size))


Epoch 1/30
375/375 [==============================] - 244s 651ms/step - loss: 0.6454 - acc: 0.6211 - val_loss: 0.5505 - val_acc: 0.7219
Epoch 2/30
375/375 [==============================] - 161s 430ms/step - loss: 0.5628 - acc: 0.7126 - val_loss: 0.4899 - val_acc: 0.7778
Epoch 3/30
375/375 [==============================] - 161s 429ms/step - loss: 0.5092 - acc: 0.7582 - val_loss: 0.4476 - val_acc: 0.7906
Epoch 4/30
375/375 [==============================] - 160s 427ms/step - loss: 0.4701 - acc: 0.7881 - val_loss: 0.4149 - val_acc: 0.8259
Epoch 5/30
375/375 [==============================] - 160s 427ms/step - loss: 0.4416 - acc: 0.8012 - val_loss: 0.4210 - val_acc: 0.8088
Epoch 6/30
375/375 [==============================] - 160s 427ms/step - loss: 0.4226 - acc: 0.8137 - val_loss: 0.3937 - val_acc: 0.8483
Epoch 7/30
375/375 [==============================] - 160s 426ms/step - loss: 0.4079 - acc: 0.8205 - val_loss: 0.4240 - val_acc: 0.7981
Epoch 8/30
375/375 [============================

In [91]:
import pandas as pd
import numpy as np
import os
import cv2

paths = ['test1/' + i for i in os.listdir('test1')]
indexes = np.array([int(file.replace('.jpg', '')) for file in os.listdir("test1")])
print(indexes[:10])
print("Test images: ", paths[:10])
images = np.array([cv2.resize(cv2.imread(img_path, cv2.COLOR_BGR2RGB), (150, 150)) for img_path in paths])
print(images[0])
print("Images arrays ", images.shape)
labels = model.predict_classes(images)
print(labels.shape)
print(labels[:10])

[    1    10   100  1000 10000 10001 10002 10003 10004 10005]
Test images:  ['test1/1.jpg', 'test1/10.jpg', 'test1/100.jpg', 'test1/1000.jpg', 'test1/10000.jpg', 'test1/10001.jpg', 'test1/10002.jpg', 'test1/10003.jpg', 'test1/10004.jpg', 'test1/10005.jpg']
[[[ 97  67  40]
  [100  65  39]
  [102  61  36]
  ...
  [207 108  76]
  [218 121  87]
  [173  84  48]]

 [[ 94  64  37]
  [ 93  57  31]
  [103  62  37]
  ...
  [207 106  81]
  [217 118  89]
  [174  81  51]]

 [[ 97  67  40]
  [100  64  38]
  [108  68  43]
  ...
  [207 106  84]
  [219 118  94]
  [182  86  59]]

 ...

 [[199 187 181]
  [209 195 189]
  [209 195 189]
  ...
  [135 142 159]
  [118 133 148]
  [108 129 145]]

 [[187 176 172]
  [198 187 183]
  [204 189 186]
  ...
  [ 94 112 110]
  [ 96 117 117]
  [ 88 111 114]]

 [[175 172 168]
  [190 182 179]
  [204 190 188]
  ...
  [193 184 195]
  [129 138 148]
  [ 70  94 102]]]
Images arrays  (12500, 150, 150, 3)
(12500, 1)
[[1]
 [0]
 [1]
 [1]
 [1]
 [0]
 [0]
 [1]
 [1]
 [0]]


In [118]:
frame = pd.DataFrame({'label': labels.flatten()})
frame.index = np.array(indexes, dtype='int32')
frame.index.name='id'
frame.sort_values(by='id', inplace=True)
frame.head()

,label
id,
1,1
2,1
3,1
4,1
5,0


In [120]:
frame.to_csv("submission" + str(len(os.listdir())))